In [1]:
from constants import img_size
import numpy as np
from keras import backend as K
from keras.layers import Input, Dense, Flatten, BatchNormalization, subtract
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.regularizers import l2
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook
from IPython.display import clear_output

Using TensorFlow backend.


## Model

In [2]:
class Seamese_Model:
    def __init__(self, base_model, opts, kernel_opts=None, bias_opts=None):
        self.base_model = base_model
        self.opts = opts
        self.kernel_opts = kernel_opts
        self.bias_opts = bias_opts
    
    def kernel_initializer(self, shape, name=None):
        if (self.kernel_opts != None):
            values = np.random.normal(loc=self.kernel_opts['loc'], scale=self.kernel_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
        
    def bias_initializer(self, shape, name=None):
        if (self.bias_opts != None):
            values = np.random.normal(loc=self.bias_opts['loc'], scale=self.bias_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
    
    def get_top_model(self):
        input_1 = Input(self.opts['features_shape'])
        input_2 = Input(self.opts['features_shape'])
        
        X_1 = Flatten()(input_1)
        X_2 = Flatten()(input_2)
        X = subtract([X_1, X_2])
        X = Dense(self.opts['features_shape'][1], activation='relu', name='dense_0', kernel_regularizer=l2(1e-3))(X)
        X = BatchNormalization()(X)
        output = Dense(1, activation='sigmoid', name='log_reg', kernel_regularizer=l2(1e-3))(X)
        
        model = Model(input=[input_1, input_2], output=output)
        return model

## Supervisor

In [3]:
class Supervisor:
    # use seed only for testing
    def __init__(self, model, data_train, classes_train, filenames_train, data_dev, classes_dev, filenames_dev, seed=None):
        self.model = model
        self.data_train = data_train
        self.classes_train = classes_train
        self.filenames_train = filenames_train
        self.data_dev = data_dev
        self.classes_dev = classes_dev
        self.filenames_dev = filenames_dev
        self.seed = seed
        
    def get_pair(self, index_1, index_2):
        el_1 = np.take(self.data_train, [index_1], axis=0)
        el_2 = np.take(self.data_train, [index_2], axis=0)
        return([el_1, el_2])
    
    def get_selection_index(self, index, indices):
        selection_index = index
        while selection_index == index:
            selection_index = np.random.choice(indices, 1)[0]
        return selection_index
    
    def get_batch(self, n, data, classes):
        np.random.seed(self.seed)
        indices = np.random.choice(list(range(len(data))), size=n)
        
        pairs = []
        y = []
        
        for index in indices[:n//2]:
            selection_indices = np.argwhere(classes == classes[index]).flatten()
            selection_index = self.get_selection_index(index, selection_indices)
            pairs.append(self.get_pair(index, selection_index))
            y.append(1)
            
        for index in indices[n//2:]:
            selection_indices = np.argwhere(classes != classes[index]).flatten()
            selection_index = self.get_selection_index(index, selection_indices)
            pairs.append(self.get_pair(index, selection_index))
            y.append(0)

        return (np.array(pairs), np.array(y))
    
    def train(self, iterations, batch_size, validation_size=0, validate_every=float('inf'), learning_rate=0.0001, path=None, k=[1]):
        self.model.compile(optimizer=tf.train.AdamOptimizer(learning_rate),
                  loss='binary_crossentropy',
                  metrics=['acc'])
        
        if path != None:
            self.model.load_weights(path)
            
        for i in tqdm_notebook(range(iterations)):
            inputs, targets = self.get_batch(batch_size, self.data_train, self.classes_train)
            metrics = self.model.train_on_batch([inputs[:,0,:], inputs[:,1,:]], targets)
            if (i % validate_every == 0) & (i != 0):
                self.checkpoint(batch_size, validation_size, i, metrics, k)
    
    def get_validation_task(self, data, classes, filenames, show_filenames=False):
        index = np.random.choice(range(data.shape[0]), 1)[0]
        
        targets = np.repeat(np.take(data, [index], axis=0), data.shape[0] - 1, axis=0)
        support_set = np.delete(data, index, axis=0)
        pairs = np.stack([targets, support_set], axis=1)
        
        pair_filenames = None
        pair_classes = None
        target_class = None
        support_classes = None
        if show_filenames == True:
            target_filenames = np.repeat(np.take(filenames, [index], axis=0), data.shape[0] - 1, axis=0)
            target_class = np.take(classes, [index], axis=0)
            support_filenames = np.delete(filenames, index, axis=0)
            support_classes = np.delete(classes, index, axis=0)
            pair_filenames = np.stack([target_filenames, support_filenames], axis=1)
        
        target_y = classes[index]
        y = (np.delete(classes, index) - target_y) == 0

        return pairs.reshape(pairs.shape[0], pairs.shape[1], 1, pairs.shape[2]), y, pair_filenames, target_class, support_classes
    
    def calculate_accuracy(self, n, batch_size, data, classes, filenames, ks, verbose=False, show_filenames=False):
        incorrect_answers_count = np.zeros(len(ks))
        incorrect_filenames = np.zeros(len(ks))
        
        for i in tqdm_notebook(range(n)):
            inputs, targets, pair_filenames, target_class, support_classes = self.get_validation_task(data, classes, filenames, show_filenames)
            predictions = self.model.predict([inputs[:,0,:], inputs[:,1,:]], batch_size=batch_size)
            
            for ind, k in enumerate(ks):
                probs = np.flipud(np.sort(predictions.flatten()))[0:k]
                indices = np.flipud(np.argsort(predictions.flatten()))[0:k]
                pred_classes = np.take(support_classes, indices, axis=0)
                unique_classes = np.unique(pred_classes)

                class_to_index = {}
                for i, cl in enumerate(unique_classes):
                    class_to_index[cl] = i

                counts = np.zeros(len(unique_classes))
                sum_probs = np.zeros(len(unique_classes))
                for cl, prob in zip(pred_classes, probs):
                    counts[class_to_index[cl]] += 1
                    sum_probs[class_to_index[cl]] += prob

                pred_class = None
                max_count = 0
                max_sum_prob = 0
                for cl, count, sum_prob in zip(unique_classes, counts, sum_probs):
                    if count > max_count:
                        pred_class = cl
                        max_count = count
                        max_sum_prob = sum_prob
                    elif (count == max_count) & (sum_prob > max_sum_prob):
                        pred_class = cl
                        max_count = count
                        max_sum_prob = sum_prob

                if pred_class != target_class[0]:
                    incorrect_answers_count[ind] += 1
#                if show_filenames == True:
#                    incorrect_filenames.append(pair_filenames[index])
                
        return list(map(lambda x: 1 - x/n, incorrect_answers_count))
    
    def checkpoint(self, batch_size, validation_size, iteration, metrics, ks=[1]):
#        train_acc, incorrect_filenames_train = self.calculate_accuracy(
#            n=validation_size, 
#            batch_size=batch_size, 
#            data=self.data_train, 
#            classes=self.classes_train, 
#            filenames=self.filenames_train, 
#            ks=ks,
#        )
        dev_acc = self.calculate_accuracy(
            n=validation_size, 
            batch_size=batch_size, 
            data=self.data_dev, 
            classes=self.classes_dev, 
            filenames=self.filenames_dev, 
            ks=ks,
        )
        print('Iteration ' + str(iteration) + '. Batch metrics [loss, accuracy]:', metrics)
#        print('Iteration ' + str(iteration) + '. Train accuracy strict:', train_acc_strict)
        print('Iteration ' + str(iteration) + '. Validation accuracy strict:', dev_acc_strict)
        print('Iteration ' + str(iteration) + '. Validation accuracy (rank=' + str(rank) + '):', dev_acc_loose)
        print('---------')
        self.model.save_weights('./siamese_model_weights/weights-' + str(iteration) + '-' + str(round(dev_acc_strict, 4)) + '.hdf5')
        
    def validate(self, weights, data, classes, filenames, n=350, batch_size=128, ks=[5], verbose=False):
        self.model.load_weights(weights)
        
        acc = self.calculate_accuracy(
            n=n, 
            batch_size=batch_size, 
            data=data, 
            classes=classes, 
            filenames=filenames, 
            ks=ks,
            show_filenames=True,
            verbose=verbose,
        )
        
        return acc

In [4]:
features_train_1 = np.load('./features/res_net/features_train.npy')
features_train_2 = np.load('./features/inception/features_train.npy')
features_train_3 = np.load('./features/xception/features_train.npy')
features_dev_1 = np.load('./features/res_net/features_dev.npy')
features_dev_2 = np.load('./features/inception/features_dev.npy')
features_dev_3 = np.load('./features/xception/features_dev.npy')

features_train = np.concatenate([features_train_1, features_train_2, features_train_3], axis=1)
features_dev = np.concatenate([features_dev_1, features_dev_2, features_dev_3], axis=1)

features_train_1 = None
features_train_2 = None
features_train_3 = None
features_dev_1 = None
features_dev_2 = None
features_dev_3 = None

In [5]:
classes_train = np.load('./features/res_net/classes_train.npy')
filenames_train = np.load('./features/res_net/filenames_train.npy')
classes_dev = np.load('./features/res_net/classes_dev.npy')
filenames_dev = np.load('./features/res_net/filenames_dev.npy')

In [6]:
model = Seamese_Model(
    ResNet50, 
    opts={
        'weights': 'imagenet',
        'input_shape': (img_size, img_size, 3),
        'features_shape': (1, features_train.shape[1]),
        'pooling': 'avg',
    },
    kernel_opts={
        'loc': 0,
        'scale': 1e-2,
    },
    bias_opts={
        'loc': 0.5,
        'scale': 1e-2,
    }
)

model = model.get_top_model()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("lo...)`


In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 6144)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 6144)      0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 6144)         0           input_1[0][0]                    
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 6144)         0           input_2[0][0]                    
__________________________________________________________________________________________________
subtract_1

In [8]:
supervisor = Supervisor(
    model=model, 
    data_train=features_train, 
    classes_train=classes_train, 
    filenames_train=filenames_train,
    data_dev=features_dev, 
    classes_dev=classes_dev,
    filenames_dev=filenames_dev,
)

## Training

In [ ]:
supervisor.train(
    iterations=250000, 
    batch_size=128, 
    validation_size=350, 
    validate_every=10000, 
    learning_rate=0.00001,
    k=[1],
    path='./siamese_model_weights/weights-30000-0.42.hdf5',
 )

## Validation

In [9]:
from scipy import ndimage
from matplotlib import pyplot as plt

In [10]:
accuracies = []

for i in range(6):
    acc = supervisor.validate(
        weights='./siamese_model_weights/weights-15000-0.4457.hdf5', 
        data=features_dev, 
        classes=classes_dev, 
        filenames=filenames_dev, 
        n=350, 
        batch_size=128, 
        ks=[1, 5, 10, 20, 50],
        verbose=False,
    )
    print(acc)
    accuracies.append(acc)


[0.35999999999999999, 0.43714285714285717, 0.48857142857142855, 0.52285714285714291, 0.5]



[0.37142857142857144, 0.4028571428571428, 0.44285714285714284, 0.46285714285714286, 0.47428571428571431]



[0.37714285714285711, 0.4514285714285714, 0.48857142857142855, 0.52285714285714291, 0.47999999999999998]



[0.43142857142857138, 0.47142857142857142, 0.5, 0.51428571428571423, 0.47142857142857142]



[0.35428571428571431, 0.40857142857142859, 0.46285714285714286, 0.47714285714285709, 0.46857142857142853]



[0.39714285714285713, 0.45428571428571429, 0.48857142857142855, 0.52857142857142858, 0.52000000000000002]


In [18]:
print(np.mean(np.array(accuracies), axis=0))

[ 0.38190476  0.43761905  0.47857143  0.5047619   0.48571429]


In [ ]:
for i in range(20):
    image_1 = np.array(ndimage.imread('./data/dev_dataset/' + incorrect_filenames[i][0], flatten=False))
    image_2 = np.array(ndimage.imread('./data/dev_dataset/' + incorrect_filenames[i][1], flatten=False))
    print(incorrect_filenames[i])
    fig = plt.figure(figsize=(20,10))
    ax1 = fig.add_subplot(2,2,1)
    ax1.imshow(image_1)
    ax2 = fig.add_subplot(2,2,2)
    ax2.imshow(image_2)